In [1]:
%matplotlib inline
  
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
model_resnet50_conv = ResNet50(weights='imagenet', include_top=False)

In [3]:
for layer in model_resnet50_conv.layers:
    layer.trainable = False

In [4]:
img_width, img_height = 96,96
train_data_dir = 'LandmarkDataset/training_Set'
val_data_dir = 'LandmarkDataset/test_Set'
model_weights_file = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
nb_train_samples = 647227
nb_val_samples = 133868
nb_epochs = 40

In [5]:
input = Input(shape=(img_width, img_height, 3))
output_resnetv50_conv = model_resnet50_conv(input)
x = Flatten()(output_resnetv50_conv)
x = Dense(64, activation='relu')(x)
#x = Dense(32, activation='relu')(x)
x = Dense(14916, activation='softmax')(x)
model = Model(input=input, output=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

In [ ]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, validation_data=validation_generator, nb_val_samples=nb_val_samples)

print('Training Completed!')